# Part 2: Content scraping 

This notebook contains the code for scraping the articles' contents, including the title and text.

In [ ]:
import time
import json
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from lxml import etree
import xmltodict
import urllib3
from selenium.common.exceptions import InvalidSessionIdException
from selenium.common.exceptions import WebDriverException

In [ ]:
# list contains all the scraped URLs
data = [] 

# loads all the URLs into the list
for i in range(0,38001,2000):
    with open(f"huffpost_article_start={i}_end={i+2000}.json","r") as file:
        data += json.load(file)

len(data)

In [ ]:
# opens Chrome browser
browser = webdriver.Chrome(ChromeDriverManager().install())
browser.maximize_window()
time.sleep(1)

In [ ]:
# list contains all the scraped content
huffPost_articles = []

# checkpoint saved for every 2000 articles scraped 
checkpoint = 2000
count = 0

In [ ]:

for url in data[0:]:

    # opens URL in the browser
    try:
        browser.get(url)
    except (InvalidSessionIdException, WebDriverException) as err :
        browser = webdriver.Chrome(ChromeDriverManager().install())
        browser.maximize_window()
        time.sleep(1)
        browser.get(url)
    
    # collects the HTML code of the website 
    try:
        s = bs(browser.page_source)
    except WebDriverException:
        browser = webdriver.Chrome(ChromeDriverManager().install())
        browser.maximize_window()
        time.sleep(1)
        browser.get(url)
        s = bs(browser.page_source)

    # each article is stored as a dictionary with 2 keys corresponding to the title and the text
    article = {}

    # collects title 
    try:
        article['title'] = s('h1',{'class','headline'})[0].get_text().strip()
    except IndexError:
        continue

    # collects text
    content = ""
    for i in s('div',{'class','primary-cli cli cli-text'}):
        try:
            content += i('p')[0].get_text().strip()
        except IndexError:
            continue 
    article['text'] = content

    # appends article dictionary to the article list
    huffPost_articles.append(article) 

    time.sleep(4)

    # once checkpoint is passed, then the article list is saved as a json file
    count += 1
    if count % checkpoint == 0:
        with open(f"huffPost_article_{count}.json","w",encoding="utf8") as file:
            json.dump(huffPost_articles, file, ensure_ascii=False)
        print(f"Checkpoint {count} passed-------")
        time.sleep(10)

In [ ]:
# checks for number of articles in the article list
len(huffPost_articles)

Note: the checkpoint algorithm was not used consistently as the algorithm was not run continuously. The sheer amount of data meant that saving manually using the code below was safer

In [ ]:
# back up file saving option
with open("backup.json","w",encoding="utf-8") as file:
    json.dump(huffPost_articles, file, ensure_ascii=False)